# numpy.linalg.svd

### Here I'm playing with the numpy svd equation to solve the long axis of a point cloud (in this case, a cell volume).

In [1]:
import numpy as np
import napari

%gui qt 
viewer = napari.Viewer()

/Users/bementmbp/opt/anaconda3/envs/napari_apoc/lib/python3.9/site-packages/napari_tools_menu/__init__.py:168: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [3]:
# load the cellpose segmentation masks
segmenations_path = './220624_Fix_Flvw_Emb_PI_Utr647_E02-10X-Z01_downsample_log_seg-cyto.npy'
segmentations = np.load(segmenations_path, allow_pickle=True).item()
all_masks = segmentations['masks']

# get the 3d coordinates for every point contained within the "82" mask
pc = np.column_stack(np.where(all_masks == 82))

# Calculate the mean of the points, i.e. the 'center' of the cloud
datamean = pc.mean(axis=0)

# shuffle the point cloud and then subsample it
np.random.shuffle(pc)
pc_subsampled = pc[::20]
pc_subsampled_mean = pc_subsampled.mean(axis=0)

In [9]:
viewer.add_points(pc, name='point cloud', opacity=0.05, size=1, face_color='green', blending='additive')
viewer.add_points(datamean, name='pc mean', opacity=0.75, size=3, face_color='green', blending='additive')

viewer.add_points(pc_subsampled, name='point cloud subsampled', opacity=0.25, size=1, face_color='red', blending='additive')
viewer.add_points(pc_subsampled_mean, name='pc subsampled mean', opacity=0.75, size=3, face_color='red', blending='additive')

<Points layer 'pc subsampled mean' at 0x7fb71c359c70>

In [48]:
uu, dd, vv = np.linalg.svd(pc_subsampled - pc_subsampled_mean)
linepts = vv[0] * np.mgrid[-100:100:2j][:, np.newaxis]
# shift by the mean to get the line in the right placeac
linepts += pc_subsampled_mean
viewer.add_shapes(linepts, shape_type='line', name='long axis', blending='additive')

<Shapes layer 'long axis' at 0x7fb758a10040>

### Let's make a couple different lines based on a couple different subsamples of the point cloud and calculate the angle difference between them

### Evolution of different array shapes:

In [46]:

b = np.mgrid[-50:50:2j][:, np.newaxis]
print(f'b is shaped {b.shape}')
print(f'b is \n{b}')
print(f'vv0 is shaped {vv[0].shape}')
print(f'vv0 is \n{vv[0]}')
answer = vv[0] * b
print(f'answer is shaped {answer.shape}')
print(f'answer is \n{answer}')
print(f'subsampled mean is shaped {pc_subsampled_mean.shape}')
print(f'subsampled mean is \n{pc_subsampled_mean}')
answer += pc_subsampled_mean
print(f'answer is shaped {answer.shape}')
print(f'answer is \n{answer}')

b is shaped (2, 1)
b is 
[[-50.]
 [ 50.]]
vv0 is shaped (3,)
vv0 is 
[-0.32028026 -0.32659726  0.88924394]
answer is shaped (2, 3)
answer is 
[[ 16.01401304  16.32986305 -44.46219697]
 [-16.01401304 -16.32986305  44.46219697]]
subsampled mean is shaped (3,)
subsampled mean is 
[ 50.70954163 209.56688494 177.90692236]
answer is shaped (2, 3)
answer is 
[[ 66.72355467 225.89674799 133.44472539]
 [ 34.69552859 193.23702189 222.36911932]]
